In [109]:
!pip install nipype jedi pypet2bids

  Using cached pypet2bids-1.0.4-py3-none-any.whl.metadata (1.2 kB)
INFO: pip is looking at multiple versions of pypet2bids to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following versions that require a different python version: 0.0.0 Requires-Python >=3.7,<3.11; 0.0.1 Requires-Python >=3.7,<3.11; 0.0.10 Requires-Python >3.7.1,<3.10; 0.0.11 Requires-Python >3.7.1,<3.10; 0.0.12 Requires-Python >3.7.1,<3.10; 0.0.13 Requires-Python >3.7.1,<3.10; 0.0.14 Requires-Python >3.7.1,<3.10; 0.0.15 Requires-Python >3.7.1,<3.10; 0.0.2 Requires-Python >=3.7,<3.11; 0.0.3 Requires-Python >=3.7,<3.11; 0.0.4 Requires-Python >3.7.1,<3.10; 0.0.4 Requires-Python >=3.7,<3.11; 0.0.5 Requires-Python >=3.7,<3.11; 0.0.6 Requires-Python >3.7.1,<3.10; 0.0.7 Requires-Python >3.7.1,<3.10; 0.0.8 Requires-Python >3.7.1,<3.10; 0.0.9 Requires-Python >3.7.1,<3.10; 1.0.0 Requires-Python >3.7.1,<3.10; 1.0.10 Requires-Python >=3.8,<3.11; 1.0.12 Requires-Python 

In [110]:
# import the relevant Python packages
import numpy
import nibabel
import nipype
import matplotlib
import subprocess

## download Phantom ZIP and extract

In [111]:
!wget -O PHANTOMS.zip https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip
!unzip -o PHANTOMS.zip 

--2024-05-29 17:30:52--  https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip
Resolving openneuropet.s3.amazonaws.com (openneuropet.s3.amazonaws.com)... 3.5.29.70, 3.5.10.193, 3.5.2.169, ...
Connecting to openneuropet.s3.amazonaws.com (openneuropet.s3.amazonaws.com)|3.5.29.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115449025 (110M) [application/zip]
Saving to: ‘PHANTOMS.zip’

PHANTOMS.zip        100%[===================>] 110.10M  54.5MB/s    in 2.0s    

2024-05-29 17:30:54 (54.5 MB/s) - ‘PHANTOMS.zip’ saved [115449025/115449025]

Archive:  PHANTOMS.zip
  inflating: __MACOSX/._OpenNeuroPET-Phantoms  
  inflating: OpenNeuroPET-Phantoms/.DS_Store  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._.DS_Store  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._sourcedata  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._sub-SiemensHRRTNRU  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._code  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._sub-G

In [112]:
import subprocess
check_for_tree = subprocess.run(['which', 'tree'], capture_output=True)
if check_for_tree.returncode == 0:
    pass
else:
    import platform
    operating_system = platform.system()
    if operating_system == 'Linux':
        subprocess.run("apt-get install tree -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install tree", shell=True)
    else:
        print("You're on your own windows user.")

In [113]:
!tree OpenNeuroPET-Phantoms/ --filelimit 15

OpenNeuroPET-Phantoms/
├── README
├── code
│   ├── matlab_conversions.m
│   └── python_conversions.sh
├── dataset_description.json
├── sourcedata
│   ├── GeneralElectricAdvance-NIMH
│   │   ├── 2d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   ├── 3d375_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   ├── 3d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   └── long_trans  [35 entries exceeds filelimit, not opening dir]
│   ├── GeneralElectricSignaPETMR-NIMH  [89 entries exceeds filelimit, not opening dir]
│   ├── SiemensBiographPETMR-NIMH
│   │   ├── AC_TOF  [150 entries exceeds filelimit, not opening dir]
│   │   ├── CT  [148 entries exceeds filelimit, not opening dir]
│   │   └── NAC  [148 entries exceeds filelimit, not opening dir]
│   └── SiemensHRRT-JHU
│       └── Hoffman.v
├── sub-GeneralElectricAdvanceLongNIMH
│   └── pet
│       ├── sub-GeneralElectricAdvanceLongNIMH_pet.json
│       └── sub-GeneralElectricAdvanc

In [114]:
# check for dcm2niix
import platform
import os
from pathlib import Path

check_dcm2niix = subprocess.run("which dcm2niix", shell=True, capture_output=True)
if check_dcm2niix.returncode == 0:
    print('dcm2niix is installed')
    version = subprocess.run('dcm2niix --version', shell=True, capture_output=True).stdout.decode()
    print(version)
    # set dcm2niix path as this is running in an ipython notebook
    dcm2niix_path = subprocess.run("which dcm2niix", shell=True, capture_output=True)
    subprocess.run(f"dcm2niix4pet --set-dcm2niix-path {dcm2niix_path.stdout.decode()}", shell=True)
else:
    print('dcm2niix is not installed')
    operating_system = platform.system()

    if operating_system == 'Linux':
        dcm2niix_install_dir = Path("dcm2niix_install")
        print(f"dcm2niix_install_dir {dcm2niix_install_dir}")
        if not dcm2niix_install_dir.exists():
            os.mkdir(dcm2niix_install_dir)
        subprocess.run("curl -fLO https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20230411/dcm2niix_lnx.zip", 
                         cwd=dcm2niix_install_dir, 
                         shell=True)
        subprocess.run("unzip dcm2niix*.zip", 
                         shell=True, 
                         cwd=dcm2niix_install_dir)
        if str(dcm2niix_install_dir) not in os.environ.get('PATH', ''):
            os.environ['PATH'] += os.pathsep + str(dcm2niix_install_dir.resolve())
        # ensure it's on the path
        check_dcm2niix_on_path = subprocess.run('dcm2niix --version && which dcm2niix', shell=True, capture_output=True)
        check_dcm2niix_on_path.stdout.decode()      
    elif operating_system == 'Darwin':
        subprocess.run("brew install dcm2niix", shell=True)
    else:
        print("You're on your own windows user.")

dcm2niix is installed
Chris Rorden's dcm2niiX version v1.0.20211006  Clang12.0.5 ARM (64-bit MacOS)
v1.0.20211006



In [115]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder

2024-05-29 17:30:58,857 - pypet2bids - WARNING - dcm2niix found on system path, but dcm2niix path is also set in ~/.pet2bidsconfig. Defaulting to dcm2niix path set in config at /usr/local/bin/dcm2niix (dcm2niix4pet.py:364)
2024-05-29 17:30:58,860 - pypet2bids - WARNING - Minimum version v1.0.20220720 of dcm2niix is recommended, found installed version v1.0.20211006 at /usr/local/bin/dcm2niix. (dcm2niix4pet.py:217)


In [116]:
!tree ./mynewfolder

./mynewfolder
├── PET_Brain_AC_TOF_resbrain_20210504071146_3.json
└── PET_Brain_AC_TOF_resbrain_20210504071146_3.nii.gz

0 directories, 2 files


In [117]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder2 --kwargs TimeZero=ScanStart Manufacturer=Siemens ManufacturersModelName=Biograph InstitutionName="NIH Clinical Center" BodyPart=Phantom Units=Bq/mL TracerName=none TracerRadionuclide=F18 InjectedRadioactivity=81.24 SpecificRadioactivity=13019.23 ModeOfAdministration=infusion FrameTimesStart=0 AcquisitionMode="list mode" ImageDecayCorrected=true ImageDecayCorrectionTime=0 AttenuationCorrection=MR-corrected FrameDuration=300 FrameTimesStart=0

2024-05-29 17:30:59,820 - pypet2bids - WARNING - dcm2niix found on system path, but dcm2niix path is also set in ~/.pet2bidsconfig. Defaulting to dcm2niix path set in config at /usr/local/bin/dcm2niix (dcm2niix4pet.py:364)
2024-05-29 17:30:59,823 - pypet2bids - WARNING - Minimum version v1.0.20220720 of dcm2niix is recommended, found installed version v1.0.20211006 at /usr/local/bin/dcm2niix. (dcm2niix4pet.py:217)


## validate BIDS dataset
https://www.npmjs.com/package/bids-validator

In [118]:
!pip install bids_validator

In [119]:
from bids_validator import BIDSValidator

In [120]:
BIDSValidator().is_bids('.') # not up to date, use browser version for now

False

## run only if freesurfer is installed

In [121]:
!mkdir PETprocessing

mkdir: PETprocessing: File exists


In [122]:
!cd PETprocessing

In [123]:
!if [ -d "PET_pipelines" ]; then pushd PET_pipelines && git pull; else git clone https://github.com/openneuropet/PET_pipelines.git; fi

Already up to date.


In [124]:
!cd PET_pipelines/pyPetSurfer 

In [125]:

# note wget fails to run on macos/zsh
#!wget https://www.dropbox.com/sh/69dwtnv29wd7jlx/AADnw5FvAANpvzKAxVQTnyhBa?dl=0
!chmod +x ds001421-1.4.1.sh
!mkdir ds001421-download/
!cp ds001421-1.4.1.sh ds001421-download/ && cd ds001421-download/ && ./ds001421-1.4.1.sh
!rm ds001421-1.4.1.sh && cd ..

mkdir: ds001421-download/: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   719  100   719    0     0   3237      0 --:--:-- --:--:-- --:--:--  3253
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   229  100   229    0     0   2974      0 --:--:-- --:--:-- --:--:--  3013
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   958  100   958    0     0   8604      0 --:--:-- --:--:-- --:--:--  8630
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   364  100   364    0     0   2616      0 --:--:-- --:--:-- --:--:--  2618
  % Total    

In [126]:
!ls PET_pipelines/pyPetSurfer

LICENSE       __init__.py   example.py
README.md     __pycache__   pet_surfer.py


In [127]:
#!pip install mri_coreg

In [128]:
!cd PET_pipelines/pyPetSurfer && python3 example.py

bids_dir: /Users/galassiae/Projects/outreach/Neurohackademy2022/ds001421-download

All computations have been successfully executed.



In [129]:
!ls .

Neurohackademy_Ganz.pdf       PHANTOMS.zip
OpenNeuroPET-Phantoms         README.md
PET_BIDS_tutorial_Colab.ipynb __MACOSX
PET_BIDS_tutorial_final.ipynb ds001421-download
PET_pipelines                 mynewfolder
PETprocessing                 mynewfolder2
